# GPTChallenge: diagnóstico a partir de HCE

Vamos a trabajar con el corpus CodEsp (textos de historial clínico etiquetados con sus códigos CIE-10 Diagnóstico)

In [2]:
import pandas as pd
import os, re
import numpy as np

pd.options.display.max_colwidth = None

In [5]:
#los códigos están en un TSV con un código por línea
train_diag = pd.read_csv("data/train/train.tsv", sep="\t", header=None, names=["archivo", "codigo"])
train_diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8316 entries, 0 to 8315
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   archivo  8316 non-null   object
 1   codigo   8316 non-null   object
dtypes: object(2)
memory usage: 130.1+ KB


In [6]:
train_diag['codigo'].value_counts()

codigo
r52        163
r69        150
r50.9      142
i10        116
r59.9       95
          ... 
d37.030      1
c79.71       1
g25.0        1
l76.3        1
n81.2        1
Name: count, Length: 2194, dtype: int64

In [7]:
#cogemos la categoría superior de cada código y las agrupamos
train_diag['cat'] = train_diag['codigo'].str.extract(r'(\w\d\d)')
print(train_diag['cat'].value_counts())
train_diag['cat'].nunique()

cat
r52    163
r10    163
r59    160
r69    150
r50    144
      ... 
c31      1
d62      1
s53      1
s34      1
n81      1
Name: count, Length: 918, dtype: int64


918

In [8]:
categories=train_diag['cat'].value_counts()[:10]
top_categorias = categories.index.to_list()
print(top_categorias)

['r52', 'r10', 'r59', 'r69', 'r50', 'r60', 'i10', 'r11', 'n28', 'd49']


In [9]:
#seleccionamos sólo las etiquetas de este subconjunto
train_diag = train_diag[np.isin(train_diag['cat'], top_categorias)]

In [11]:
#cargamos los dos conjuntos de train
path = 'data/train/text_files/'

corpus = []
for f in [f for f in os.listdir(path) if f.endswith('.txt')]:
    with open(os.path.join(path, f), encoding="utf8") as text:
        texto = text.read()
    #buscamos códigos
    file = f[:-4]
    codigos = train_diag.query('archivo==@file')['cat'].to_list()
    codigos = list(set(codigos))
    if codigos:
        corpus.append({
            'archivo': file,
            'texto': texto,
            'codigos': codigos
        })
    
df_train = pd.DataFrame(corpus).set_index('archivo')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 562 entries, S0004-06142005000700014-1 to S2340-98942015000100005-1
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   texto    562 non-null    object
 1   codigos  562 non-null    object
dtypes: object(2)
memory usage: 13.2+ KB


In [12]:
df_train.sample(3)

,texto,codigos
archivo,,
S0210-48062004000500008-1,"Paciente de 29 años de edad que acude al Servicio de Urgencias de nuestro Hospital ante la presencia de una erección levemente dolorosa de diez horas de evolución. El paciente refiere haber presentado cuatro episodios similares en los últimos tres meses pero en todos ellos la erección había cedido como máximo en tres o cuatro horas.\nEl paciente niega el antecedente de traumatismo peneano, el uso de drogas u otros fármacos incluidas inyecciones intracavernosas. Así mismo no presenta ningún otro síntoma acompañante como astenia, anorexia o fiebre.\nSe realiza una gasometría de la sangre extraída de los cuerpos cavernosos obteniendo resultados analíticos de sangre venosa (pCO2 163,1 mmHg, HCO3 13,2 mEq/l plasma).\nAnte este priapismo de prolongada duración y la ausencia de indicios que nos hagan pensar en un priapismo de alto flujo, procedemos a intentar revertir la erección con la mayor brevedad posible, con el fin de calmar el dolor y evitar un daño irreversible en los cuerpos cavernosos.\nSe punciona con un catéter intravenoso de 19 G en ambos cuerpos cavernosos, aspirando sangre en repetidas ocasiones y se deja que ésta gotee. Tras este primer paso no se obtiene disminución de la erección, por lo que se prepara una solución de fenilefrina (agonista alfaadrenérgico) en suero salino a una concentración de 0,1 mg/ml. Se inyecta 2 ml de la solución en un cuerpo cavernoso, comprimiendo posteriormente. Se repite nuevamente esta maniobra en el cuerpo cavernoso contralateral. Ante la ausencia de respuesta, se realiza una nueva solución de fenilefrina, mezclando 10 mg de fenilefrina en 11 de suero salino y se van introduciendo 20-30 ml de esta solución en cada uno de los cuerpos cavernosos para lavarlos. Tras la repetición en varias ocasiones de esta última maniobra, se consigue obtener finalmente flaccidez peneana.\nDurante su permanencia en urgencias y tras lograr flaccidez, se extraen analíticas sanguíneas que revelan una marcada elevación de las cifras de leucocitos (414 x 109/L) y plaquetas (1100 x 109/L) en sangre.\nAnte la sospecha de un síndrome mieloproliferativo se consulta con el Servicio de Hematología, que diagnostica leucemia mieloide crónica.\n\nPosteriormente el paciente es sometido a un trasplante de médula ósea alogénico de donante familiar con HLA idéntico. La evolución de su enfermedad hematológica es favorable quedando el cuadro de priapismo totalmente resuelto al tratar la leucemia mieloide crónica y sin alteración en la potencia sexual.\n","[r69, r50, r52]"
S0004-06142006000300015-1,"Paciente de 50 años con antecedente de litiasis renal de repetición que consultó por hematuria recidivante y sensación de malestar. El estudio citológico seriado de orinas demostró la presencia de células atípicas sospechosas de malignidad. Los estudios de imagen permitieron observar una lesión heterogénea que ocupaba la práctica totalidad de la pelvis renal derecha. Se realizó nefrectomía total y ureterectomía parcial observándose un riñón de 10.5 x 5 cm de dimensiones máximas que mostraba a nivel de pelvis renal una lesión irregular y heterogénea de tonalidad pardusca en cuyo seno destacaban áreas hemorrágicas y otras ligeramente amarillentas y brillantes de consistencia sumamente blanda al corte. Dicha lesión, relativamente bien delimitada, no invadía el parénquima ni el seno renal quedando limitada al sistema pielocalicial aunque comprimiendo con márgenes expansivos la corteza en uno de los polos renales. El estudio histológico demostró que la lesión estaba constituida por nidos tumorales bien delimitados sin evidencia de invasión del parénquima renal, englobando una matriz mucinosa central la cual constituía más del 50% del volumen tumoral. En el seno de dicha matriz se encontraron flotando células neoplásicas sueltas y nidos de menor tamaño que los situados a nivel periférico. Las células tumorales mostraban morfología poligonal con núcleos centrales sin nucleolo aparente y 

## Cargar los textos del conjunto de test

## Binarizar las etiquetas

In [13]:
# para entrenar un clasificador multi-etiqueta generamos una matriz binaria de las etiquetas
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(df_train['codigos'])

#Guardamos las clases utilizadas en el conjunto de train
clases = mlb.classes_
num_classes = clases.shape
print(num_classes[0])

10


## Procesamiento del lenguaje natural

## Modelos

## Guardar predicciones de Test